<a href="https://colab.research.google.com/github/meets5800/EmbeddedSystems/blob/main/Lab03_TensorFlow_vs_PyTorch_MEET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 03: TensorFlow vs. PyTorch
- Train a model on MNIST in both TensorFlow and PyTorch, convert to TFLite and ONNX.  
- Use tf.GradientTape for Tensorflow custom training loop.



## TensorFlow Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and normalize the MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalization factor for grayscale images
x_test = x_test / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),         # MNIST images are 28x28
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),  # 64 hidden neurons
    tf.keras.layers.Dense(10, activation='softmax')  # 10 output classes for digits 0-9
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',      # Suitable for one-hot encoded labels
              metrics=['accuracy'])

# Train the model and measure time
start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()

print(f"TF Training time: {end - start:.2f} seconds")  # Output training time

# Evaluate the model
model.evaluate(x_test, y_test)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.8626 - loss: 0.4858
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9558 - loss: 0.1542
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9688 - loss: 0.1085
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9761 - loss: 0.0799
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9801 - loss: 0.0662
TF Training time: 38.01 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9707 - loss: 0.1005


[0.08727740496397018, 0.973800003528595]

## Convert TensorFlow model to TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmp8usjq8lh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  137514791917456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137514791920144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137514791920336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137514791920528: TensorSpec(shape=(), dtype=tf.resource, name=None)


## PyTorch Implementation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Transform: flatten the 28x28 images to 784-dimensional vectors
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten to 784
])

# Data loaders
train_loader = DataLoader(
    datasets.MNIST(root='./data', train=True, transform=transform, download=True),
    batch_size=32
)
test_loader = DataLoader(
    datasets.MNIST(root='./data', train=False, transform=transform, download=True),
    batch_size=1000
)

# Define the network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)     # 784 input features to 64 hidden units
        self.fc2 = nn.Linear(64, 10)      # 64 hidden units to 10 output classes
    def forward(self, x):
        x = F.relu(self.fc1(x))           # Activation after first layer
        return self.fc2(x)                # Output logits from second layer

model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

# Training
start = time.time()
for epoch in range(5):
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

# Evaluation
model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()
print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")


100%|██████████| 9.91M/9.91M [00:00<00:00, 40.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.06MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.65MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.91MB/s]


PyTorch Training time: 55.50 seconds
Test accuracy: 0.9649


## Convert PyTorch model to ONNX

In [ ]:
# Install ONNX
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 87.6 MB/s eta 0:00:00


In [ ]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalize pixel values to [0, 1]
x_test = x_test / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets
batch_size = 32         # Same batch size as in earlier TF/PyTorch examples
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),          # Input shape for MNIST images
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),   # 64 neurons, ReLU activation
    tf.keras.layers.Dense(10, activation='softmax') # 10 neurons, Softmax for classification
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.5599, Accuracy: 0.0938
Step 100, Loss: 0.8005, Accuracy: 0.7348
Step 200, Loss: 0.2844, Accuracy: 0.7990
Step 300, Loss: 0.3592, Accuracy: 0.8303
Step 400, Loss: 0.4234, Accuracy: 0.8491
Step 500, Loss: 0.1734, Accuracy: 0.8629
Step 600, Loss: 0.1543, Accuracy: 0.8717
Step 700, Loss: 0.3524, Accuracy: 0.8775
Step 800, Loss: 0.1531, Accuracy: 0.8840
Step 900, Loss: 0.4796, Accuracy: 0.8888
Step 1000, Loss: 0.3706, Accuracy: 0.8924
Step 1100, Loss: 0.2733, Accuracy: 0.8961
Step 1200, Loss: 0.0857, Accuracy: 0.8991
Step 1300, Loss: 0.3952, Accuracy: 0.9009
Step 1400, Loss: 0.2163, Accuracy: 0.9038
Step 1500, Loss: 0.1115, Accuracy: 0.9060
Step 1600, Loss: 0.2847, Accuracy: 0.9085
Step 1700, Loss: 0.2273, Accuracy: 0.9106
Step 1800, Loss: 0.3915, Accuracy: 0.9127
Training Accuracy for epoch 1: 0.9143

Epoch 2/5
Step 0, Loss: 0.0655, Accuracy: 0.9688
Step 100, Loss: 0.0433, Accuracy: 0.9527
Step 200, Loss: 0.3096, Accuracy: 0.9456
Step 300, Loss: 0.0480, Accuracy:

## Performance Otimization with Graph Execution using @tf.function

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalize pixel values to [0, 1]
x_test = x_test / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),         # MNIST images are 28x28
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),  # 64 hidden neurons
    tf.keras.layers.Dense(10, activation='softmax')# 10 output classes with softmax
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compile the function into a graph for faster execution
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3941, Accuracy: 0.0938
Step 100, Loss: 0.3854, Accuracy: 0.7432
Step 200, Loss: 0.4176, Accuracy: 0.8099
Step 300, Loss: 0.2513, Accuracy: 0.8367
Step 400, Loss: 0.4739, Accuracy: 0.8551
Step 500, Loss: 0.3649, Accuracy: 0.8663
Step 600, Loss: 0.1463, Accuracy: 0.8756
Step 700, Loss: 0.5474, Accuracy: 0.8833
Step 800, Loss: 0.2559, Accuracy: 0.8887
Step 900, Loss: 0.1649, Accuracy: 0.8936
Step 1000, Loss: 0.5758, Accuracy: 0.8974
Step 1100, Loss: 0.2143, Accuracy: 0.9010
Step 1200, Loss: 0.0831, Accuracy: 0.9039
Step 1300, Loss: 0.1452, Accuracy: 0.9061
Step 1400, Loss: 0.1728, Accuracy: 0.9080
Step 1500, Loss: 0.1822, Accuracy: 0.9103
Step 1600, Loss: 0.2649, Accuracy: 0.9130
Step 1700, Loss: 0.0902, Accuracy: 0.9149
Step 1800, Loss: 0.2435, Accuracy: 0.9169
Training Accuracy for epoch 1: 0.9183

Epoch 2/5
Step 0, Loss: 0.0927, Accuracy: 1.0000
Step 100, Loss: 0.0236, Accuracy: 0.9548
Step 200, Loss: 0.2071, Accuracy: 0.9541
Step 300, Loss: 0.0646, Accuracy: